In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/DataSets')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /DataSets


In [0]:
!ls "/DataSets/My Drive/DataSets" 

'Assignment 1.zip'  'Assignment 2.zip'


In [0]:
import zipfile as zf
files = zf.ZipFile("/DataSets/My Drive/DataSets/Assignment 1.zip", 'r')
files.extractall('/DataSetExtracted')
files.close()

In [0]:
import os
os.listdir('/DataSetExtracted')

['Assignment 1.docx', 'dogs-vs-cats.zip']

In [0]:
import zipfile as zf
files = zf.ZipFile('/DataSetExtracted/dogs-vs-cats.zip', 'r')
files.extractall('/DataSetExtracted/Dogs-vs-cats/')
files.close()

In [0]:
data_dir=('/DataSetExtracted/Dogs-vs-cats/')

In [0]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
# Hyper parameters
num_epochs = 2
num_classes = 2
batch_size = 10
learning_rate = 0.001

In [0]:
# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(180),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=10)

In [0]:
# Convolutional neural network (two convolutional layers)
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size=(5, 5), stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(5, 5), stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=(3, 3), padding=1)
        self.fc1 = nn.Linear(in_features= 64 * 6 * 6, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=2)
        
        
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2)
        
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2)
        
        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2)
        
#         print(X.shape)
        X = X.view(X.shape[0], -1)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        
#         X = torch.sigmoid(X)
        return X

In [0]:
import torch
torch.cuda.current_device()
torch.cuda.device(0)
torch.cuda.is_available()
torch.cuda.device_count()
torch.cuda.is_available()

True

In [0]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [0]:
 # Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
model = Net(num_classes).to(device)

In [0]:
model = Net(num_classes).cuda()

In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [10/901], Loss: 0.7907
Epoch [1/2], Step [20/901], Loss: 0.7214
Epoch [1/2], Step [30/901], Loss: 0.6993
Epoch [1/2], Step [40/901], Loss: 0.6645
Epoch [1/2], Step [50/901], Loss: 0.5513
Epoch [1/2], Step [60/901], Loss: 0.6770
Epoch [1/2], Step [70/901], Loss: 0.5540
Epoch [1/2], Step [80/901], Loss: 0.6554
Epoch [1/2], Step [90/901], Loss: 0.6929
Epoch [1/2], Step [100/901], Loss: 0.6297
Epoch [1/2], Step [110/901], Loss: 0.6621
Epoch [1/2], Step [120/901], Loss: 0.6184
Epoch [1/2], Step [130/901], Loss: 0.7423
Epoch [1/2], Step [140/901], Loss: 0.6228
Epoch [1/2], Step [150/901], Loss: 0.5819
Epoch [1/2], Step [160/901], Loss: 0.6467
Epoch [1/2], Step [170/901], Loss: 0.4878
Epoch [1/2], Step [180/901], Loss: 0.6829
Epoch [1/2], Step [190/901], Loss: 0.6726
Epoch [1/2], Step [200/901], Loss: 0.7188
Epoch [1/2], Step [210/901], Loss: 0.7239
Epoch [1/2], Step [220/901], Loss: 0.5222
Epoch [1/2], Step [230/901], Loss: 0.6743
Epoch [1/2], Step [240/901], Loss: 0.5702
E

In [0]:
Predictions=[]
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        Predictions.append(predicted)
         
    print(len(Predictions))
    print(len(Predictions[0]))
    print(Predictions[0])
    
from sklearn.metrics import classification_report
val_preds = np.argmax(Predictions, axis=-1)
print(classification_report(labels, val_preds))
 

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')


100
10
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')


RuntimeError: ignored

In [0]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 53.6 %
